In [1]:
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from tqdm import tqdm

import numpy as np
import progressbar
import torch
import xarray as xr


from data.satellite import SatelliteData
import utils

%matplotlib inline 

In [2]:
import torchvision
from models import dcgan_64, vgg_64

# Try using a pretrained VGG

In [3]:
vgg = torchvision.models.vgg16_bn()

In [7]:
vgg_64.encoder(dim=3)

encoder(
  (c1): Sequential(
    (0): vgg_layer(
      (main): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (1): vgg_layer(
      (main): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
  )
  (c2): Sequential(
    (0): vgg_layer(
      (main): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (1): vgg_layer(
      (main): Sequential(
        (0): Conv2d(128, 128, ker

In [24]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

Band info for reference: https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l1c/

In [2]:
kwargs = {
    "data_root": "../Arizona-processed4",
    "seq_len": 60,
    "no_randomization": True
}
train_data = SatelliteData(train=True, **kwargs)
test_data = SatelliteData(train=False, **kwargs)

Using: 2168 for training
Using: 556 for testing


In [3]:
past = 53
future = 7

In [4]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        return self.linear(x)

In [5]:
img_size = 32
input_dim  = 5*3*img_size*img_size
output_dim = 7*3*img_size*img_size
model = linearRegression(input_dim, output_dim)

In [9]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [10]:
model.double()

linearRegression(
  (linear): Linear(in_features=15360, out_features=21504, bias=True)
)

In [12]:
epochs = 10
for epoch in tqdm(range(epochs)):
    for i in range(len(train_data)):
        data = train_data[i]
        X = data[past-5:past]
        Y = data[past:]
        for v1, v2, h1, h2 in [(0, 32, 0, 32), (0, 32, 32, 64), (32, 64, 0, 32), (32, 64, 32, 64)]:
            X_quadrant = X[:, :, v1:v2, h1:h2].flatten()
            Y_quadrant = Y[:, :, v1:v2, h1:h2].flatten()
            optimizer.zero_grad()
            Y_pred = model(X_quadrant)
            
            loss = criterion(Y_pred, Y_quadrant)
            print(loss)
            loss.backward()
            optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]

tensor(3.5466, dtype=torch.float64, grad_fn=<MseLossBackward>)


  0%|          | 0/10 [00:01<?, ?it/s]

tensor(2.4735, dtype=torch.float64, grad_fn=<MseLossBackward>)


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 2642411520 bytes. Error code 12 (Cannot allocate memory)


In [4]:
X_test = []
Y_test = []

for i in tqdm(range(len(test_data))):
    values = test_data[i].numpy()
    X_test.append(values[past-12:past])
    Y_test.append(values[past:])

100%|██████████| 556/556 [00:38<00:00, 14.31it/s]


In [5]:
X_train = []
Y_train = []

# Using every third item to save memory
for i in tqdm(range(0, len(train_data), 4)):
    values = train_data[i].numpy()
    X_train.append(values[past-12:past])
    Y_train.append(values[past:])

100%|██████████| 542/542 [00:35<00:00, 15.14it/s]


In [ ]:
# Create linear regression object
regr = SGDRegressor()

# Train the model using the training sets
regr.fit(X_train, Y_train)

# Make predictions using the testing set
Y_pred = regr.predict(X_test)